<a href="https://colab.research.google.com/github/saeedghadiri/IranHolidaysEDA/blob/main/Iran'sHolidaysEDA.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Download Data
first we need to download data. I created a project for fetching this data from http://time.ir in https://github.com/saeedghadiri/IranHolidays

In [ ]:
!wget https://raw.githubusercontent.com/saeedghadiri/IranHolidays/master/iran_holidays.csv

--2022-01-26 13:07:15--  https://raw.githubusercontent.com/saeedghadiri/IranHolidays/master/iran_holidays.csv
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.111.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 153113 (150K) [text/plain]
Saving to: ‘iran_holidays.csv.1’

iran_holidays.csv.1 100%[===================>] 149.52K  --.-KB/s    in 0.02s   

2022-01-26 13:07:15 (6.45 MB/s) - ‘iran_holidays.csv.1’ saved [153113/153113]



# Import Packages

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px

sns.set_style('dark')

# Utils for persian text and months

In [ ]:
j_months_fa = [
    u'فروردین',
    u'اردیبهشت',
    u'خرداد',
    u'تیر',
    u'اَمرداد',
    u'شهریور',
    u'مهر',
    u'آبان',
    u'آذر',
    u'دی',
    u'بهمن',
    u'اسفند',
]

In [ ]:
def replace_text(word):
    replacements = {'پ': '\u067e', 'چ': '\u0686', 'ج': '\u062c', 'ح': '\u062d', 'خ': '\u062e', 'ه': '\u0647',
                    'ع': '\u0639', 'غ': '\u063a', 'ف': '\u0641', 'ق': '\u0642', 'ث': '\u062b', 'ص': '\u0635',
                    'ض': '\u0636', 'گ': '\u06af', 'ک': '\u06a9', 'م': '\u0645', 'ن': '\u0646', 'ت': '\u062a',
                    'ا': '\u0627', 'ل': '\u0644', 'ب': '\u0628', 'ي': '\u06cc', 'س': '\u0633', 'ش': '\u0634',
                    'و': '\u0648', 'ئ': '\u0626', 'د': '\u062f', 'ذ': '\u0630', 'ر': '\u0631', 'ز': '\u0632',
                    'ط': '\u0637', 'ظ': '\u0638', 'ژ': '\u0698', 'آ': '\u0622', 'ي': '\u064a', '؟': '\u061f',
                    'ك': '\u06a9', 'ي': 'ی'}
    for src, target in replacements.items():
        word = word.replace(src, target)

    return word


# Read and Add Features

In [ ]:

df = pd.read_csv('iran_holidays.csv')
df['date'] = pd.to_datetime(df['date'])

del df['Unnamed: 0']
df['week_day'] = df['date'].dt.day_name() # add day of week to the dataframe
df['year'] = df['fa_date'].apply(lambda x: str(x)[:4]) # add year to the dataframe
df['month'] = df['fa_date'].apply(lambda x: str(x)[4:6]) # add month
df['day'] = df['fa_date'].apply(lambda x: str(x)[6:]) # add day
df.head()
df['reason'] = df['reason'].apply(replace_text)


In [ ]:
df['reason'] = df['reason'].str.replace('\d+', '') # replace numbers
for m in j_months_fa:
  df['reason'] = df['reason'].str.replace(m, '') # replace every month with empty
df['reason'] = df['reason'].str.strip()
df['reason'] = df['reason'].replace('\s+', ' ', regex=True)

In [ ]:
list(df['reason'].unique())

['جشن نوروز/جشن سال نو',
 'عیدنوروز',
 'روز جمهوری اسلامی',
 'جشن سیزده به در',
 'شهادت حضرت علی علیه السلام[ رمضان ]',
 'عید سعید فطر[ شوال ]',
 'تعطیل به مناسبت عید سعید فطر[ شوال ]',
 'شهادت امام جعفر صادق علیه السلام[ شوال ]',
 'رحلت حضرت امام خمینی',
 'قیام',
 'عید سعید قربان[ ذوالحجه ]',
 'عید سعید غر خم[ ذوالحجه ]',
 'تاسوعای حسینی[ محرم ]',
 'عاشورای حسینی[ محرم ]',
 'اربعین حسینی[ صفر ]',
 'رحلترسول اکرم؛شهادت امام حسن مجتبی علیه السلام[ صفر ]',
 'شهادت امام رضا علیه السلام[ صفر ]',
 'شهادت امام حسن عسکری علیه السلام[ ربیع الاول ]',
 'میلاد رسول اکرم و امام جعفر صادق علیه السلام[ ربیع الاول ]',
 'شهادت حضرت فاطمه زهرا سلام الله علیها[ جما الثانیه ]',
 'ولادت امام علی علیه السلام و روز پدر[ رجب ]',
 'مبعث رسول اکرم (ص)[ رجب ]',
 'پیروزی انقلاب اسلامی',
 'ولادت حضرت قائم عجل الله تعالی فرجه و جشن نیمه شعبان[ شعبان ]',
 'روز ملی شدن صنعت نفت ایران',
 'آخرین روز سال']

In [ ]:
df_count = df.groupby('year')['fa_date'].count()
print('total days: {}  total years: {} mean: {} var: {}'.format(len(df) , len(df['year'].unique()) , df_count.mean(), df_count.var()))

total days: 1694  total years: 61 mean: 27.770491803278688 var: 0.5797814207650266


# Effective Dates

But there are holidays that overlap with weekends, and even worse overlap with each other.

In [ ]:
df_real = df.groupby('date').first()
df_real = df_real[(df_real['week_day'] != 'Thursday') & (df_real['week_day'] != 'Friday')]

df_count_real = df_real.groupby('year')['fa_date'].count()
print('total holidays that are real: {} which is only {} of total mean: {} var: {}'.format(len(df_real), len(df_real)/len(df) , df_count_real.mean(), df_count_real.var()))

total holidays that are real: 1185 which is only 0.6995277449822904 of total mean: 19.42622950819672 var: 4.248633879781419


In [ ]:
df_weekday = df_real['week_day'].value_counts()
df_weekday = df_weekday.to_frame().reset_index()
df_weekday.columns = ['week_day', 'count']

fig = px.bar(df_weekday, x="week_day", y="count", text="count", title="Number of holidays in each week day from 1370 to 1430")
fig.update_layout(autosize=False, width=450, height=500, title={'y':0.9, 'x':0.5, 'xanchor': 'center', 'yanchor': 'top', 'font' : {'size': 15}}, template="plotly_dark")
fig.update_traces(marker_color='yellow')
fig.show()

In [ ]:

df_yearly = df_real['year'].value_counts()
df_yearly = df_yearly.to_frame().reset_index()

df_yearly.columns = ['year', 'count']
df_yearly = df_yearly.sort_values(by='count')

fig = px.bar(df_yearly, x="year", y="count", text="count", title="Number of holidays in each year from 1370 to 1430")
fig.update_layout(autosize=False, width=1000, height=500, xaxis=dict(tickmode='linear'), title={'y':0.9, 'x':0.5, 'xanchor': 'center', 'yanchor': 'top'}, template="plotly_dark")
fig.update_traces(marker_color='brown')
fig.show()

In [ ]:
df_monthly = df_real['month'].value_counts()
df_monthly = df_monthly.to_frame().reset_index()

df_monthly.columns = ['month', 'count']
df_monthly = df_monthly.sort_values(by='count')

fig = px.bar(df_monthly, x="month", y="count", text="count", title="Number of holidays in each month from 1370 to 1430")
fig.update_layout(autosize=False, width=800, height=500, xaxis=dict(tickmode='linear'), title={'y':0.9, 'x':0.5, 'xanchor': 'center', 'yanchor': 'top'}, template="plotly_dark")
fig.update_traces(marker_color='red')
fig.show()

Now count each reason for these years

In [ ]:
df_real['reason'].value_counts()

عیدنوروز                                                         132
شهادت امام جعفر صادق علیه السلام[ شوال ]                          46
شهادت امام حسن عسکری علیه السلام[ ربیع الاول ]                    46
ولادت حضرت قائم عجل الله تعالی فرجه و جشن نیمه شعبان[ شعبان ]     46
مبعث رسول اکرم (ص)[ رجب ]                                         45
روز ملی شدن صنعت نفت ایران                                        44
عاشورای حسینی[ محرم ]                                             44
شهادت حضرت فاطمه زهرا سلام الله علیها[ جما الثانیه ]              44
جشن نوروز/جشن سال نو                                              44
تاسوعای حسینی[ محرم ]                                             44
ولادت امام علی علیه السلام و روز پدر[ رجب ]                       44
رحلترسول اکرم؛شهادت امام حسن مجتبی علیه السلام[ صفر ]             44
عید سعید فطر[ شوال ]                                              43
میلاد رسول اکرم و امام جعفر صادق علیه السلام[ ربیع الاول ]        43
جشن سیزده به در                   

find the most consecutive days without holidays in each year

In [ ]:
start_date = df_real.index[0]
df_real['day_from_start'] = (df_real.index -  start_date) # first count days from starting date
df_real

,fa_date,reason,week_day,year,month,day,day_from_start
date,,,,,,,
1991-03-23,13700103,عیدنوروز,Saturday,1370,01,03,0 days
1991-03-24,13700104,عیدنوروز,Sunday,1370,01,04,1 days
1991-04-01,13700112,روز جمهوری اسلامی,Monday,1370,01,12,9 days
1991-04-02,13700113,جشن سیزده به در,Tuesday,1370,01,13,10 days
1991-04-07,13700118,شهادت حضرت علی علیه السلام[ رمضان ],Sunday,1370,01,18,15 days
...,...,...,...,...,...,...,...
2051-11-13,14300822,شهادت امام حسن عسکری علیه السلام[ ربیع الاول ],Monday,1430,08,22,22150 days
2051-11-22,14300901,میلاد رسول اکرم و امام جعفر صادق علیه السلام[ ...,Wednesday,1430,09,01,22159 days
2052-02-05,14301116,شهادت حضرت فاطمه زهرا سلام الله علیها[ جما الث...,Monday,1430,11,16,22234 days


In [ ]:
df_real['days_passed_from_last_holiday'] = df_real['day_from_start'] - df_real['day_from_start'].shift(1)
df_real['days_passed_from_last_holiday'] = df_real['days_passed_from_last_holiday'].dt.days - 1
df_days_passed = df_real.groupby('year').agg({'days_passed_from_last_holiday':'max'})
df_days_passed = df_days_passed.reset_index()

In [ ]:
fig = px.bar(df_days_passed, x="year", y="days_passed_from_last_holiday", text="days_passed_from_last_holiday", title="Number of cosecutive days without any holiday in each year from 1370 to 1430")
fig.update_layout(autosize=False, width=1000, height=500, xaxis=dict(tickmode='linear'), title={'y':0.9, 'x':0.5, 'xanchor': 'center', 'yanchor': 'top'}, template="plotly_dark")
fig.show()